In [1]:
import pandas as pd

from tika import parser

pd.set_option('display.max_rows', 500)

In [2]:
def text_score_to_float(text_score):
    return float(text_score.replace(",", "."))

def sort_by_specific_score(scores_df, column_name):
    base_name, factor_str = column_name.replace(")", "").split("/")
    factor = int(factor_str)
    scores_by_current_score = scores_df[column_name].sort_values(ascending=False).reset_index(level=0)
    scores_by_current_score[f"{base_name}/100)"] = scores_by_current_score[column_name] / factor * 100
    scores_by_current_score.index = scores_by_current_score.index.set_names(["Rank"]) + 1
    
    return scores_by_current_score

### Ejecución del servidor tika

Si el servidor no se ha ejecutado, descomentar la siguiente línea para iniciar tika. Es necesario tener instalado Java y que el sistema reconozca el comando `java` (para lo cual debe estar agregado al PATH).

In [3]:
#!java -jar tika-server-1.24.jar

## Lectura de datos

El archivo de puntajes de hoja de vida para el Centro Industrial de Mantenimiento Integral (Girón, Santander) está publicado en el blog oficial del centro, más precisamente en [esta página](https://centroindustrialmantenimientointegral.blogspot.com/2021/11/publicacion-verificacion-hoja-de-vida.html). 

El documento de Word publicado en dicha página, con los puntajes de las hojas de vida, está alojado en OneDrive y puede consultarse [aquí](https://onedrive.live.com/redir?resid=6F159E97EAA13858%211096&authkey=%21AAEmIExDy4eOee4&page=View&wdEmbedFS=1).

He archivado ambas páginas, en caso de que alguna de las fuentes sea modificada:
* [Entrada en el blog](https://archive.md/rEwZg) - [respaldo](https://web.archive.org/web/20211204161526/https://centroindustrialmantenimientointegral.blogspot.com/2021/11/publicacion-verificacion-hoja-de-vida.html)
* [Listado de puntajes](https://archive.md/N9Smx) - [respaldo](https://web.archive.org/web/20211204161643/https://onedrive.live.com/View.aspx?resid=6F159E97EAA13858!1096&wdEmbedFS=1&wdo=2&authkey=!AAEmIExDy4eOee4)

En cuanto a los resultados generales, fueron obtenidos de la [página de concursos y convocatorias de la ESAP](https://www.esap.edu.co/portal/index.php/concursos-y-convocatorias-2/). Allí han sido publicados los resultados preliminares de las pruebas; [éste es el enlace de descarga directa](https://www.esap.edu.co/portal/index.php/Descargas/3238/sena/61237/result-prueba-impr-03122021.pdf).

También he archivado la página de la ESAP con los resultados; el enlace de descarga puede verificarse pasando el puntero sobre el botón de descarga y observando la esquina inferior izquierda (o, utilizando la opción copiar dirección del enlace):
* [Página ESAP](https://archive.md/Y1Hls) - [respaldo](https://web.archive.org/web/20211204161658/https://www.esap.edu.co/portal/index.php/concursos-y-convocatorias-2/)

In [4]:
parsed_test_results_pdf = parser.from_file("Result-Prueba-Impr-03122021.pdf")
parsed_giron_cvs_pdf = parser.from_file("PUBLICACIÓN DE RESULTADOS CIMI (1).pdf")

### Limpieza de datos

Transforma los "No presentó prueba" a `0` y remueve el nombre del centro de los datos relevantes para que sea más fácil leerlos.

In [5]:
test_results_data = parsed_test_results_pdf["content"]
test_results_data = test_results_data.replace("No presentó prueba", "0")

giron_csv_data = parsed_giron_cvs_pdf["content"]
giron_csv_data = giron_csv_data.replace("Centro Industrial de Mantenimiento Integral ", "")

### Lectura de Datos - Girón

Lee el pdf con los puntajes de las hojas de vida de Girón, conserva únicamente los datos relevantes y los guarda en un diccionario cuyas llaves son los `national_id`s (en su mayoría cédulas de ciudadanía) mapeados al puntaje de la hoja de vida.

In [6]:
giron_scores = {}

for giron_csv_datum in giron_csv_data.split("\n"):
    split_line = giron_csv_datum.split()[1:]
    if len(split_line) == 2 and split_line[0].isnumeric():
        national_id = split_line[0]
        cv_score = split_line[1]
        giron_scores[national_id] = text_score_to_float(cv_score)

###  Lectura de datos - Puntajes de prueba (ESAP)

Lee el pdf con los puntajes de las pruebas (habilidades digitales y socioemocional), conserva los datos relevantes y los agrega a un diccionario cuyas llaves son los `national_id`s mapeados en una tupla a los puntajes obtenidos, transformados en valores de punto flotante (i.e. números con decimales).

In [7]:
test_result_scores = {}

for test_result_datum in test_results_data.split("\n"):
    split_line = test_result_datum.split()[1:]
    if len(split_line) == 3 and split_line[0].isnumeric():
        national_id = split_line[0]
        scores = (
            text_score_to_float(split_line[1]),
            text_score_to_float(split_line[2])
        )
        test_result_scores[national_id] = scores

### Cálculo de Puntajes

Toma los datos anteriores, itera los puntajes para Girón (relevantes para este caso) y para cada persona (basado en su `national_id`) calcula el puntaje total y los puntajes por cada ítem estandarizados al rango correspondiente:
* Hojas de vida: 0 a 40.
* Prueba habilidades digitales (`test_1`): 0 a 20.
* Prueba socioemocional (`test_2`): 0 a 40.

Los resultados son conservados en un nuevo diccionario mapeado por `national_id` a una tupla que contiene como primer elemento el puntaje total, seguido de los puntajes descritos anteriormente, en ese orden.

In [8]:
total_scores = {}

for national_id in giron_scores:
    cv_score = giron_scores[national_id] * 0.4
    test_1_score = test_result_scores[national_id][0] * 0.2
    test_2_score = test_result_scores[national_id][1] * 0.4
    total = cv_score + test_1_score + test_2_score
    total_scores[national_id] = (total, cv_score, test_1_score, test_2_score)

### Presentación de Resultados

Convierte el diccionario con los resultados a un `DataFrame` de pandas, indizado por `national_id`.

In [9]:
scores_df_columns = [
    "Total score (/100)",
    "CV sore (/40)",
    "Test 1 score (/20)",
    "Test 2 score (/40)"
]
scores_df = pd.DataFrame.from_dict(total_scores, orient="index", columns=scores_df_columns)
scores_df.index = scores_df.index.set_names(["National ID"])

#### Puntaje total

Resultados organizados por puntaje total.

In [10]:
scores_by_total = scores_df.sort_values(by=scores_df_columns[0], ascending=False).reset_index(level=0)
scores_by_total.index = scores_by_total.index.set_names(["Rank"]) + 1
scores_by_total

,National ID,Total score (/100),CV sore (/40),Test 1 score (/20),Test 2 score (/40)
Rank,,,,,
1,91485661,83.5104,33.7104,15.000,34.800
2,91228781,82.7340,38.0000,14.334,30.400
3,23181732,78.1916,34.4576,13.334,30.400
4,13706489,77.3980,32.0000,15.666,29.732
5,63490103,76.5340,29.6000,15.334,31.600
6,63509285,76.4752,29.9432,14.000,32.532
7,91420169,76.3340,32.0000,13.666,30.668
8,91487046,76.1556,30.0236,14.000,32.132
9,91281598,76.0940,28.1600,15.666,32.268


#### Puntaje hoja de vida

Resultados organizados por puntaje en la hoja de vida.

In [11]:
column_cv = scores_df_columns[1]

scores_by_cv = sort_by_specific_score(scores_df, column_cv)
scores_by_cv

,National ID,CV sore (/40),CV sore (/100)
Rank,,,
1,91228781,38.0000,95.000
2,23181732,34.4576,86.144
3,91485661,33.7104,84.276
4,13465771,32.8000,82.000
5,13716299,32.0772,80.193
6,91264734,32.0000,80.000
7,91275647,32.0000,80.000
8,91420169,32.0000,80.000
9,91241521,32.0000,80.000


#### Puntaje prueba de habilidades digitales

Resultados organizados por puntaje en la prueba de habilidades digitales. Se presentan dos columnas: una con el puntaje estandarizado y la otra con el puntaje en escala 0 a 100.

In [12]:
column_test_1 = scores_df_columns[2]

scores_by_test_1 = sort_by_specific_score(scores_df, column_test_1)
scores_by_test_1

,National ID,Test 1 score (/20),Test 1 score (/100)
Rank,,,
1,1095822478,17.000,85.00
2,1098705285,17.000,85.00
3,63511924,16.666,83.33
4,91159510,16.334,81.67
5,1098649274,16.334,81.67
6,63507562,16.334,81.67
7,13956532,16.334,81.67
8,91541184,16.000,80.00
9,1098755264,16.000,80.00


#### Puntaje prueba de habilidades socioemocionales

Resultados organizados por puntaje en la prueba de habilidades socioemocionales. Se presentan dos columnas: una con el puntaje estandarizado y la otra con el puntaje en escala 0 a 100.

In [13]:
column_test_2 = scores_df_columns[3]

scores_by_test_2 = sort_by_specific_score(scores_df, column_test_2)
scores_by_test_2

,National ID,Test 2 score (/40),Test 2 score (/100)
Rank,,,
1,91485661,34.800,87.00
2,91517000,34.400,86.00
3,5622430,33.200,83.00
4,1098755264,33.068,82.67
5,91541184,33.068,82.67
6,1098670142,32.932,82.33
7,1095822478,32.800,82.00
8,37727082,32.668,81.67
9,1098649274,32.668,81.67


### Almacenamiento de resultados

Conserva los resultados en un archivo de Excel, con cada criterio de clasificación en una hoja de cálculo apropiadamente nombrada.

In [14]:
with pd.ExcelWriter('ResultadosSENA2022.xlsx') as writer:
    scores_by_total.to_excel(writer, sheet_name="Generales")
    scores_by_cv.to_excel(writer, sheet_name="Hoja de Vida")
    scores_by_test_1.to_excel(writer, sheet_name="Prueba Habilidades Digitales")
    scores_by_test_2.to_excel(writer, sheet_name="Prueba Socioemocional")

Los resultados anteriores están sujetos a las reclamaciones que reciba la ESAP (que, sin conocer las respuestas, no deberían llevar a modificar sensiblemente los puntajes). Además, las posiciones no toman en cuenta a qué cargo está aplicando cada participante pues esta información no es pública a la fecha

La intención de este análisis es contribuir a que el proceso sea más transparente. Es fácil modificar el código aquí contenido para analizar otros centros o para obtener los puntajes al 60% de todos los aplicantes del país.